## Content

1. [Machine Translation](#1.-Machine-Translation)
2. [Neural Machine Translation](#2.-Neural-Machine-Translation)
3. [Loading the dataset](#3.-Loading-the-dataset)
4. [Preparing the Text Data](#4.-Preparing-the-Text-Data)
5. [Build, Train Model](#5.-Build,-Train-Model)
6. [Evaluate Model](#6.-Evaluate-Model)

## 1. Machine Translation

When we hear about language translation, the first thing that comes to mind is Google Translate!!!, it's such a life savior. It has changed the world by allowing people to communicate when it wouldn’t otherwise be possible.<br>
Machine translation, sometimes referred to by the abbreviation MT is a sub-field of computational linguistics that investigates the use of software to translate text or speech from one language to another.(Source - Wikipedia)<br><br>
***
As famously quoted by the late Susan Sontag,<br>
> _"Translation is the circulatory system of the world's literatures"_
***


Machine translation is the task of automatically converting source text in one language to text in another language. Given a sequence of text in a source language, there is no one single best translation of that text to another language. This is because of the natural ambiguity and flexibility of human language. It is a challenging task that traditionally involves large statistical models developed using highly sophisticated linguistic knowledge.<br>
**Classical machine translation** methods often involve rules for converting text in the source language to the target language. The rules are often developed by linguists and may operate at the lexical, syntactic, or semantic level. This focus on rules gives the name to this area of study: Rule-based Machine Translation, or RBMT. The key limitations of the classical machine translation approaches are both the expertise required to develop the rules, and the vast number of rules and exceptions required.<br>


## 2. Neural Machine Translation

Neural machine translation, or NMT for short, is the use of neural network models to learn a statistical model for machine translation. The key benefit to the approach is that a single system can be trained directly on source and target text, no longer requiring the pipeline of specialized systems used in statistical machine learning.<br>
As such, neural machine translation systems are said to be _end-to-end systems_ as only one model is required for the translation.


### Encoder-Decoder Model

Multilayer Perceptron neural network models can be used for machine translation, although the models are limited by a fixed-length input sequence where the output must be the same length.<br>
These early models have been greatly improved upon recently through the use of recurrent neural networks(RNNs) organized into an encoder-decoder architecture that allow for variable length input and output sequences.<br>
As stated in [Neural Machine Translation by Jointly Learning to Align and Translate, 2014](https://arxiv.org/abs/1409.0473),
>An encoder neural network reads and encodes a source sentence into a fixed-length vector. A decoder then outputs a translation from the encoded vector. The whole encoder–decoder system, which consists of the encoder and the decoder for a language pair, is jointly trained to maximize the probability of a correct translation given a source sentence.

Key to the encoder-decoder architecture is the ability of the model to encode the source text into an internal fixed-length representation called the context vector. Interestingly, once encoded, different decoding systems could be used, in principle, to translate the context into different languages.

### Encoder-Decoders with Attention

Although effective, the Encoder-Decoder architecture has problems with long sequences of text to be translated. The problem stems from the fixed-length internal representation that must be used to decode each word in the output sequence.<br>
The solution is the use of an attention mechanism that allows the model to learn where to place attention on the input sequence as each word of the output sequence is decoded.<br>
> Using a fixed-sized representation to capture all the semantic details of a very long sentence is very difficult. A more efficient approach, however, is to read the whole sentence or paragraph, then to produce the translated words one at a time, each time focusing on a different part of the input sentence to gather the semantic details required to produce the next output word.

The encoder-decoder recurrent neural network architecture with attention is currently the state-of-the-art on some benchmark problems for machine translation. And this architecture is used in the heart of the Google Neural Machine Translation system, or GNMT, used in their Google Translate service.Although effective, the neural machine translation systems still suffer some issues, such as scaling to larger vocabularies of words and the slow speed of training the models.

We will develop a neural machine translation system for translating German phrases to English. 

## 3. Loading the Dataset

The dataset is available on [ManyThings.org](http://www.manythings.org/anki/) which comprises of German phrases and their English counterparts.<br><br>
Dataset: German–English deu-eng.zip<br>

In [ ]:
#Unzipping the file and putting it in destination folder
!unzip 'data/deu-eng.zip' -d 'data/'

Unzipping the file returns a file _deu.txt_ that has pairs of English to German phrases, one per line with a tab separating the language. We will frame the prediction problem as given a sequence of words in German as input, translate or predict the sequence of words in English.

## 4. Preparing the Text Data

The next step is to prepare the text data ready for modeling. It's very important to know the steps we will be performing as far as data cleaning process is concerned since it can vary depending on the datasets.<br>
Few observations from the dataset we have at our disposal,
- Symbols like punctuation and other special characters
- The text contains uppercase and lowercase.
- There are duplicate phrases in English with different translations in German.
- The file is ordered by sentence length with very long sentences toward the end of the file.

A good text cleaning procedure may handle some or all of these observations. Data preparation is divided into two subsections:
- Clean Text
- Split Text into Train, Test dataset(s)

### Clean Text

In [16]:
#Importing libraries
import string
import re
from pickle import dump
from unicodedata import normalize
import numpy as np

First, we must load the data in a way that preserves the Unicode German characters. The function below called `load_doc()` will load the file as a blob of text.<br>
Each line contains a single pair of phrases, first English and then German, separated by a *tab character*. We must split the loaded text by line and then by phrase. The function `to_pairs()` will split the loaded text.<br><br>
We are now ready to clean sentence. The specific cleaning operations we will perform are as follows:
- Remove all non-printable characters.
- Remove all punctuation characters.
- Normalize all Unicode characters to ASCII (e.g. Latin characters).
- Normalize the case to lowercase.
- Remove any remaining tokens that are not alphabetic.

We will perform these operations on each phrase for each pair in the loaded dataset. The `clean_pairs()` function below implements these operations.<br>

Finally, now that the data has been cleaned, we can save the list of phrase pairs to a file ready for use. The function `save_clean_data()` uses the pickle API to save the list of clean text to file.


In [19]:
#Load doc into memory
def load_doc(filename):
    #open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    #read all text
    text = file.read()
    #close the file
    file.close()
    return text

#Split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split('\t') for line in lines]
    return pairs

# clean a list of lines
def clean_pairs(lines):
    cleaned = list()
    #prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    #prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return np.array(cleaned)
                       
#Save a list of clean sentences to a file
def save_clean_data(sentences, filename):
    dump(sentences, open(filename, 'wb'))
    print('Saved: %s' % filename)


Time to load the dataset and apply the above defined-function(s).

In [20]:
#Load Dataset
filename = 'data/deu.txt'
doc = load_doc(filename)
#Split into English-German pairs
pairs = to_pairs(doc)
#Clean sentences
clean_pairs = clean_pairs(pairs)
#Save clean pairs to file
save_clean_data(clean_pairs, 'data/english-german.pkl')
#Verifying if the functions work as intended
for i in range(100):
    print('[%s]=> [%s]' % (clean_pairs[i,0], clean_pairs[i,1]))
    

Saved: data/english-german.pkl
[hi]=> [hallo]
[hi]=> [gru gott]
[run]=> [lauf]
[wow]=> [potzdonner]
[wow]=> [donnerwetter]
[fire]=> [feuer]
[help]=> [hilfe]
[help]=> [zu hulf]
[stop]=> [stopp]
[wait]=> [warte]
[go on]=> [mach weiter]
[hello]=> [hallo]
[i ran]=> [ich rannte]
[i see]=> [ich verstehe]
[i see]=> [aha]
[i try]=> [ich probiere es]
[i won]=> [ich hab gewonnen]
[i won]=> [ich habe gewonnen]
[smile]=> [lacheln]
[cheers]=> [zum wohl]
[freeze]=> [keine bewegung]
[freeze]=> [stehenbleiben]
[got it]=> [kapiert]
[got it]=> [verstanden]
[got it]=> [einverstanden]
[he ran]=> [er rannte]
[he ran]=> [er lief]
[hop in]=> [mach mit]
[hug me]=> [druck mich]
[hug me]=> [nimm mich in den arm]
[hug me]=> [umarme mich]
[i fell]=> [ich fiel]
[i fell]=> [ich fiel hin]
[i fell]=> [ich sturzte]
[i fell]=> [ich bin hingefallen]
[i fell]=> [ich bin gesturzt]
[i know]=> [ich wei]
[i lied]=> [ich habe gelogen]
[i lost]=> [ich habe verloren]
[i paid]=> [ich habe bezahlt]
[i paid]=> [ich zahlte]
[i sang

### Split Text into Train, Test dataset(s)

The clean data contains a little over 195,000 phrase pairs and some of the pairs towards the end of the file are very long.<br>
This is a good number of examples for developing a small translation model. The complexity of the model increases with the number of examples, length of phrases, and size of the vocabulary.<br>
Although we have a good dataset for modeling translation, we will simplify the problem slightly to dramatically reduce the size of the model required, and in turn the training time required to fit the model.<br>
We will simplify the problem by reducing the dataset to the first 10,000 examples in the file; these will be the shortest phrases in the dataset. Further, we will consider the first 9,000 of those as examples for training and the remaining 1,000 examples to test the fit model.

In [ ]:
sdsfghj

## 5. Build, Train Model

## 6. Evaluate Model